<div class="alert alert-block alert-info">

- [Import the needed libraries](#2nd-bullet)<br>
- [Import the dataset](#3rd-bullet)<br>
    
- [Filter methods](#6th-bullet)<br>
    - [Univariate variables](#7th-bullet)<br>
    - [Correlation Indices](#8th-bullet)<br>
    - [Chi-Square](#9th-bullet)<br>
- [Wrapper Methods](#10th-bullet)<br>
    - [RFE](#11th-bullet)<br>
        - [RFE Logistic Reg.](#12th-bullet)<br>
        - [RFE XGBoostClassifier.](#12th-bullet)<br>
- [Embedded Methods](#13th-bullet)<br>
    - [Lasso](#14th-bullet)<br>
- [Final Insights](#15th-bullet)<br>
    
</div>

<a class="anchor" id="2nd-bullet">

### Import the needed libraries
    
</a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set() # setting seaborn default for plots

# data partition
from sklearn.model_selection import train_test_split


#filter methods
# spearman
# chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE, RFECV
from xgboost import XGBClassifier

# embedded methods
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import mutual_info_regression, SelectKBest  # mutual information is a embedded method for regression problems

# model assessment
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')


<a class="anchor" id="3rd-bullet">

### Import the dataset
    
</a>

In [3]:
preproc_train = pd.read_csv('traindf (1).csv')

preproc_val = pd.read_csv('valdf (1).csv')

preproc_test = pd.read_csv('testdf (1).csv')


In [ ]:
preproc_train = pd.read_csv('traindf (1).csv')

preproc_val = pd.read_csv('valdf (1).csv')

preproc_test = pd.read_csv('testdf (1).csv')


In [13]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit the encoder on the training data
label_encoder.fit(preproc_train['Claim Injury Type'])

# Transform the 'Claim Injury Type' column in both training and validation sets
preproc_train['Claim Injury Type'] = label_encoder.transform(preproc_train['Claim Injury Type'])
preproc_val['Claim Injury Type'] = label_encoder.transform(preproc_val['Claim Injury Type'])

In [16]:
pd.set_option('display.max_columns', None)
preproc_train


,Claim Injury Type,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,First Hearing Date,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Agreement Reached,Number of Dependents,Accident Year,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,3,0.518093,-0.020370,1.353821,-0.495974,-0.028681,-2.352697,-0.575548,0.983139,46591,1.046863,-0.037533,1.098612,-1.101128,212791,23478,28947,41386,11432,0.0,1.497677,0.092321,1,1,0,1,0,0,0,0.733411,0.764416,-0.276840,1,0,0,1
1,2,-0.875110,-0.511111,1.402126,0.955044,-0.531549,0.000000,-0.494377,-0.780636,31683,1.046863,-1.477658,0.000000,1.514005,212791,18015,4764,33580,10451,0.0,-0.501860,0.632632,1,0,1,0,0,0,0,-0.210297,0.676657,-0.221387,0,0,1,1
2,1,0.078134,0.609259,0.000000,-0.133220,0.621415,0.000000,-0.209935,0.983139,42673,1.046863,-0.037533,0.000000,-0.933566,212791,2585,122836,15030,11203,0.0,-0.001976,-0.447990,1,0,0,0,0,0,1,-0.030941,0.502881,1.181511,0,0,0,0
3,1,-1.315068,0.051852,0.000000,1.317798,0.045889,0.000000,-0.625062,-1.524399,42673,1.046863,-0.037533,0.000000,-0.260367,212791,8654,88310,6312,11237,0.0,-1.001744,0.092321,1,0,0,0,0,0,1,-0.561594,-0.356220,-0.117342,0,0,0,0
4,2,-0.068519,-0.440741,1.223679,0.156984,-0.472275,-3.170124,-0.453747,-1.524399,34872,1.046863,-1.413414,0.000000,-0.417166,212791,13351,44593,33580,11003,0.0,0.497909,0.632632,1,1,1,1,0,0,1,0.125710,0.943210,0.018146,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,3,-1.241742,-0.150000,1.409856,1.245247,-0.162524,-2.651452,-0.411805,0.983139,20630,1.046863,0.396110,0.693147,1.514005,43043,3511,2257,6524,10805,0.0,1.497677,0.092321,1,1,1,1,0,0,1,-2.070147,0.038237,-0.596408,0,0,0,0
459216,3,0.884725,-0.009259,1.383011,-0.858729,-0.017208,-2.307054,-0.617431,0.983139,46591,1.046863,0.171258,1.945910,1.514005,212791,29667,88310,18167,11429,0.0,-1.001744,0.092321,1,1,1,1,0,0,1,0.999137,-0.998549,0.099896,0,0,0,0
459217,3,1.031378,-0.292593,1.361083,-1.003831,-0.309751,0.000000,-0.618653,0.983139,24655,-1.041200,-0.037533,0.000000,-1.022713,108512,9519,122836,38473,14127,0.0,0.997793,0.092321,1,0,0,0,0,0,0,1.256250,1.412170,-0.380313,0,0,0,0
459218,1,-1.241742,0.642593,0.000000,1.172696,0.655832,0.000000,-0.322674,0.983139,42673,1.046863,-0.037533,0.000000,-0.579708,212791,25481,122836,15030,11226,0.0,-1.001744,-0.447990,1,0,0,0,0,0,1,1.158953,-0.938574,0.291408,0,0,0,0


In [4]:
#how many columns are in the dataset

preproc_train.shape

(459220, 36)

In [5]:
X_preproc_train = preproc_train.drop(columns=['Claim Injury Type'])

y_preproc_train = preproc_train[['Claim Injury Type']]

X_preproc_val =	preproc_train.drop(columns=['Claim Injury Type'])

y_preproc_val =preproc_train[['Claim Injury Type']]



In [6]:

# #passar o claim identifier para o index

# y_preproc_train_copy = y_preproc_train.copy()
# y_preproc_train.set_index('Claim Identifier', inplace=True)

# y_preproc_val.set_index('Claim Identifier', inplace=True)

# X_preproc_train.set_index('Claim Identifier', inplace=True)

# preproc_test.set_index('Claim Identifier', inplace=True)

# X_preproc_val.set_index('Claim Identifier', inplace=True)


<div class="alert alert-block alert-success">

<a class="anchor" id="5th-bullet">    
    
## Feature Selection
    
</a>
    
</div>

<a class="anchor" id="6th-bullet">

## Filter methods
    
</a>

<a class="anchor" id="7th-bullet">

### Univariate variables
    
</a>

Check if any of the numerical variables is univariate (variance is equal to 0), then drop those variables if any of those exist.

In [7]:
X_preproc_train.select_dtypes(include='object')

""
0
1
2
3
4
...
459215
459216
459217
459218


from this we can tell that, there are no categorical features

In [8]:
X_preproc_train.var().sort_values()

,0
Alternative Dispute Resolution_Y,4.483018e-03
Received_C2,2.460152e-02
Agreement Reached,4.431073e-02
COVID-19 Indicator_Y,4.580129e-02
high_risk_body,4.764902e-02
high_risk_cause,1.018937e-01
high_risk_nature,1.168806e-01
market_indicator,1.436771e-01
Hearing_held,1.936208e-01
Attorney/Representative_Y,2.164144e-01


No feature to be dropped from this one

<a class="anchor" id="8th-bullet">

### Correlation indices
    
</a>

In [9]:
X_preproc_train.head()



,Age at Injury,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,0.518093,-0.020370,1.353821,-0.495974,-0.028681,-2.352697,-0.575548,0.983139,46591,1.046863,...,0,0,0,0.733411,0.764416,-0.276840,1,0,0,1
1,-0.875110,-0.511111,1.402126,0.955044,-0.531549,0.000000,-0.494377,-0.780636,31683,1.046863,...,0,0,0,-0.210297,0.676657,-0.221387,0,0,1,1
2,0.078134,0.609259,0.000000,-0.133220,0.621415,0.000000,-0.209935,0.983139,42673,1.046863,...,0,0,1,-0.030941,0.502881,1.181511,0,0,0,0
3,-1.315068,0.051852,0.000000,1.317798,0.045889,0.000000,-0.625062,-1.524399,42673,1.046863,...,0,0,1,-0.561594,-0.356220,-0.117342,0,0,0,0
4,-0.068519,-0.440741,1.223679,0.156984,-0.472275,-3.170124,-0.453747,-1.524399,34872,1.046863,...,0,0,1,0.125710,0.943210,0.018146,0,0,0,0


In [10]:
#drop binary columns
continuous_features = preproc_train.drop(columns=['COVID-19 Indicator_Y', 'Alternative Dispute Resolution_Y', 'Gender_M', 'high_risk_nature', 'Received_C2', 'Received_C3', 'Hearing_held', 'Attorney/Representative_Y', 'high_risk_body', 'high_risk_cause', 'market_indicator', 'Agreement Reached'])

In [11]:
# round to 1 decimal places
continuous_features = continuous_features.round(1)
def cor_heatmap(cor):
    plt.figure(figsize=(24,20))
    sns.heatmap(data = cor, annot = True, cmap = plt.cm.Blues, fmt='.2f')
    plt.show()

In [12]:

cor_spearman = continuous_features.corr(method ='spearman')
cor_spearman

ValueError: could not convert string to float: '4. TEMPORARY'

In [ ]:
cor_heatmap(cor_spearman)

We will assume that the Thresholds for feature relevance is 0.2 and redundance for 0,8


### Redundancy
drop 'wcio_pca1' or WCIO 'Nature of Injury Code'
drop 'c2 date' or assembly date

### Relevance



In [ ]:
def TestIndependence(X,y,var,alpha=0.05):
    dfObserved = pd.crosstab(y,X)
    chi2, p, dof, expected = stats.chi2_contingency(dfObserved.values)
    dfExpected = pd.DataFrame(expected, columns=dfObserved.columns, index = dfObserved.index)
    if p<alpha:
        result="{0} is IMPORTANT for Prediction".format(var)
    else:
        result="{0} is NOT an important predictor. (Discard {0} from model)".format(var)
    print(result)

In [ ]:
#turn into a 1D array

for var in cat_X_preproc_train.columns:
    TestIndependence(cat_X_preproc_train[var],y_preproc_train,var)


<a class="anchor" id="11th-bullet">

## Wrapper Methods
    
</a>

<a class="anchor" id="12th-bullet">

### RFE
    
</a>

In [ ]:
num_X_preproc_train.columns.value_counts().sum()

<a class="anchor" id="13th-bullet">

#### RFE Logistic Regression

</a>

Using RFE, the objective is finding the optimal number of features to select

we will use only 4000 rows of data so rfe is faster


In [ ]:
#perform crossvalidation with 10 folds


#number of features
nof_list= np.arange(1,36)
high_score=0

#Variable to store the optimum features
nof=0
train_f1score_list =[]
val_f1score_list = []

for n in range(len(nof_list)):
    model = LogisticRegression()

    rfe = RFE(estimator = model,n_features_to_select = nof_list[n])
    X_train_rfe = rfe.fit_transform(num_X_preproc_train[:40000],y_preproc_train[:40000])
    X_val_rfe = rfe.transform(num_X_preproc_val[:40000])

    model.fit(X_train_rfe[:40000],y_preproc_train[:40000])

    # Predictions for F1 score
    y_preproc_train_pred = model.predict(X_train_rfe)
    y_preproc_val_pred = model.predict(X_val_rfe)

    #storing results on training data
    train_f1score = f1_score(y_preproc_train[:40000],y_preproc_train_pred[:40000], average='macro')
    train_f1score_list.append(train_f1score)

    #storing results on training data
    val_f1score = f1_score(y_preproc_val[:40000],y_preproc_val_pred[:40000], average='macro')
    val_f1score_list.append(val_f1score)

    #check best score
    if(val_f1score >= high_score):
        high_score = val_f1score
        nof = nof_list[n]

        features_to_select = pd.Series(rfe.support_, index = num_X_preproc_train.columns)

print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))
print(f"Features to select: \n{features_to_select}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(nof_list, train_f1score_list, label="Train F1 Score")
plt.plot(nof_list, val_f1score_list, label="Validation F1 Score")
plt.xlabel("Number of Features")
plt.ylabel("F1 Score (Macro)")
plt.title("F1 Score vs Number of Features")
plt.legend()
plt.show()

### RFE XGBoosting

In [ ]:
#number of features
nof_list= np.arange(1, 40)
high_score=0

#Variable to store the optimum features
nof=0
train_f1score_list =[]
val_f1score_list = []

for n in range(len(nof_list)):

    #using XGBoost
    model = XGBClassifier()

    rfe = RFECV(estimator = model,min_features_to_select = nof_list[n], step=1, cv=5)
    X_train_rfe = rfe.fit_transform(num_X_preproc_train,y_preproc_train)
    X_val_rfe = rfe.transform(num_X_preproc_val)

    model.fit(X_train_rfe,y_preproc_train)

    # Predictions for F1 score
    y_preproc_train_pred = model.predict(X_train_rfe)
    y_preproc_val_pred = model.predict(X_val_rfe)

    #storing results on training data
    train_f1score = f1_score(y_preproc_train,y_preproc_train_pred, average='macro')
    train_f1score_list.append(train_f1score)

    #storing results on training data
    val_f1score = f1_score(y_preproc_val,y_preproc_val_pred, average='macro')
    val_f1score_list.append(val_f1score)

    #check best score
    if(val_f1score >= high_score):
        high_score = val_f1score
        nof = nof_list[n]
        features_to_select = pd.Series(rfe.support_, index = num_X_preproc_train.columns)


print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))
print(f"Features to select: \n{features_to_select}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(nof_list, train_f1score_list, label="Train F1 Score")
plt.plot(nof_list, val_f1score_list, label="Validation F1 Score")
plt.xlabel("Number of Features")
plt.ylabel("F1 Score (Macro)")
plt.title("F1 Score vs Number of Features")
plt.legend()
plt.show()

<a class="anchor" id="13th-bullet">

## Embedded Methods
    
</a>

<a class="anchor" id="14th-bullet">

### Lasso Regression
    
</a>

In [ ]:
def plot_importance(coef,name):
    imp_coef = coef.sort_values()
    plt.figure(figsize=(8,10))
    imp_coef.plot(kind = "barh")
    plt.title("Feature importance using " + name + " Model")
    plt.show()

In [ ]:
#
reg = LassoCV()

reg.fit(num_X_preproc_train, y_preproc_train)

coef = pd.Series(reg.coef_, index = num_X_preproc_train.columns)
coef

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
coef.sort_values()

In [ ]:
plot_importance(coef,'Lasso')

<a class="anchor" id="15th-bullet">

### Mutual Information Regressor
    https://scikit-learn.org/1.5/modules/generated/sklearn.feature_selection.SelectKBest.html
    https://scikit-learn.org/1.5/modules/generated/sklearn.feature_selection.mutual_info_regression.html
</a>

In [ ]:
#  Mutual Information Regression for feature selection in regression problems
mir = SelectKBest(score_func=mutual_info_regression, k= 'all')

# fit the model to the training data
mir.fit(preproc_train, y_preproc_train)

# get the scores of the features
mir_scores = pd.Series(mir.scores_, index = preproc_train.columns)

# plot the scores
plot_importance(mir_scores,'Mutual Information')

<a class="anchor" id="15th-bullet">

## Final Insights:
    
</a>

### Numerical Data

| Predictor | Spearman | RFE LR | RFE XGB | Lasso | Mutual Information Reg. | What to do? |
| --- | --- | --- | --- |--- |---|---|
| AgAge at Injurye | Keep | Keep | Keep |Keep | Keep | Include in the model |
| Gender_M | Keep | Keep | Keep |Keep| Keep | Include in the model |
| Gender_F | Keep| Discard | Discard |Keep ?| Keep |  Discard |
| Alternative Dispute Resolution_Y | Keep | Keep | Keep | Keep |Keep | Include in the model |
| Alternative Dispute Resolution_N | Keep? | Discard | Keep | Keep | Keep | Keep |
| COVID-19 Indicator_Y | Keep? | Discard | Discard | Keep | Keep | Discard |
| Hearing_held | Discard |Discard |Keep | Keep? |  Keep | Try with and without |
| Received_C3 | Discard |Discard |Keep | Keep? |  Keep | Try with and without |
| Received_C2 | Discard |Discard |Keep | Keep? |  Keep | Try with and without |
| Agreement Reached | Keep |Discard |Keep | Keep? |  Keep | Try with and without |
| Gender_U | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Accident Year | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| WCIO Part Of Body Code | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| WCIO Nature of Injury Code | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| WCIO Cause of Injury Code  | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Medical Fee Region | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Industry Code | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Carrier Type | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Birth Year | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Number of Dependents | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Zip Code | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| IME-4 Count  | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Attorney/Representative_Y | Keep |Discard |Keep | Keep? | Keep |  Try with and without |
| Average Weekly Wage | Keep |Discard |Keep | Keep? | Keep |  Try with and without |


<hr>


